# FastAI + PyJet code to generate the tabular data with the ResNet-34 predictions (env: FastAI-V1 + PyJet)

## !! Important: to run this code first need to install FastAI following the steps on https://github.com/fastai/fastai
- create an enviorement using:  \$ conda create -n FastAI-V1 python=3.6 conda=4.6.14
- load the enviorement: \$ conda activate FastAI-V1
- install fastai using: (FastAI-V1)\$ conda install -c pytorch -c fastai fastai
- removing the jpeg rubish library: (FastAI-V1)\$ conda uninstall --force jpeg libtiff -y
- installing Pillow: (FastAI-V1)\$ conda install -c conda-forge libjpeg-turbo pillow==6.0.0
- run this: (FastAI-V1)\$ CC="cc -mavx2" pip install --no-cache-dir -U --force-reinstall --no-binary :all: --compile pillow-simd

### !!! After install FastAI, it's necessary to install the pytables to read the h5 files with pandas:
- (FastAI-V1)\$ conda install pytables

the PyJet library:
- (FastAI-V1)\$ pip install --user pyjet

### and you are good to go:

In [ ]:
from __future__ import division 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fastai.vision import *
from tqdm import tqdm
import fastai
import gc

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib as mpl
from pyjet import cluster,DTYPE_PTEPM
import pandas as pd
from tqdm import tqdm
import os
from itertools import combinations 

In [ ]:
## Disable the interactive session from matplotlib to avoid fill up all your memory:
mpl.interactive(False)

In [ ]:
# load the pre-trained ResNet-34 model to be use to get the predictions (need for bdt):
learn = load_learner('./ResNet_models/','resnet_34_LSC_kaiming_uni_10_mdl.pkl')

In [ ]:
# Option 3: Use generator to loop over the whole file
def generator(filename, chunksize=512,total_size=1100000):

    m = 0
    
    while True:

        yield pd.read_hdf(filename,start=m*chunksize, stop=(m+1)*chunksize)

        m+=1
        if (m+1)*chunksize > total_size:
            m=0


In [ ]:
fn =  '/home/felipe/LHCOlympics_2020/data/events_LHCO2020_BlackBox1.h5'

In [ ]:
fb = generator(fn,chunksize=512*100)

In [ ]:
class DetectorImage:
    '''
    Class for generate pixeleted jet images from pseudojets objects from pyjet.
    arguments:
        pixel_size = size of the pixels based onthe hist2d bin images
        image_size = dimension for the images generated
        eta_min = min pseudo-rapidity
        eta_max = max pseudo-rapidity
        path = path to save the images
    '''
    
    def __init__(self, pixel_size=200, image_size=200, 
                 eta_min= -5., eta_max=5., path=''):
        self.eta_min, self.eta_max = eta_min, eta_max
        self.extent = self.eta_min, self.eta_max, -np.pi, np.pi
        self.image_size = image_size
        self.bins = pixel_size

        self.eta_edges = np.linspace(self.eta_min, self.eta_max, self.bins + 1)
        self.phi_edges = np.linspace(-np.pi, np.pi, self.bins + 1)
        self.eta = np.linspace(self.eta_min, self.eta_max, self.bins + 1)[:-1] + (self.eta_max - self.eta_min) / (2 * self.bins)
        self.phi = np.linspace(-np.pi, np.pi, self.bins + 1)[:-1] + (np.pi / self.bins)

        self.phi_min,self.phi_max = np.min(self.phi), np.max(self.phi)
        self.path = path
        
    def pseudojets_images(self, pseudojet, weight, idx):
        '''
        Function to drawn jet images from the pyjet pseudojets objects.
        arg:
            pseudojet = unclustered pseudojet
            weight = weights arguments for the histo2d function to colorize the pixels
            idx = event index
        '''
        fig, ax = plt.subplots(subplot_kw=dict(xlim=(self.eta_min,self.eta_max),
                                               ylim=(self.phi_min,self.phi_max)),
                               figsize=(self.image_size/10, self.image_size/10),dpi=10)
        ax.axis('off')

        hadrons, _, _ = np.histogram2d(pseudojet[:]['eta'], pseudojet[:]['phi'], 
                                   bins=(self.eta_edges, self.phi_edges), weights=weight)
        ax.imshow(np.ma.masked_where(hadrons == 0, hadrons).T,
                  extent=self.extent, aspect=(self.eta_max - self.eta_min) / (2*np.pi),
                  interpolation='none', origin='lower')


        plt.tight_layout()
        plt.savefig(self.path + '/{}.png'.format(idx), dpi=10)
        plt.clf()
        plt.close('all')

    def image_name(idx):
        return '{}.png'.format(idx)

In [ ]:
def comp_delta_phi(phi1,phi2):
    delta_phi = np.abs(phi1 - phi2)
    if delta_phi > np.pi:
            delta_phi = 2. * np.pi - delta_phi
    return delta_phi

In [ ]:
def delta_R_sub(subjets, n_subjets):
    lj = np.zeros([n_subjets], dtype=DTYPE_PTEPM)
    if len(subjets) < n_subjets:
        for i in range(len(subjets)):
            lj[i]['pT'] = subjets[i].pt
            lj[i]['eta'] = subjets[i].eta
            lj[i]['phi'] = subjets[i].phi
    else:
        for i in range(n_subjets):
            lj[i]['pT'] = subjets[i].pt
            lj[i]['eta'] = subjets[i].eta
            lj[i]['phi'] = subjets[i].phi
        
    comb = combinations(lj, 2)
    deltaR = []
    for i in comb:
        deleta = i[0]['eta'] - i[1]['eta']
        deltaphi = comp_delta_phi(i[0]['phi'], i[1]['phi'])
        deltaR.append(np.sqrt((deleta)**2 + (deltaphi)**2))
    return deltaR

In [ ]:
batch_idx = 0
for batch in fb:
    events_combined = batch.T
    img_base_path = '/home/felipe/LHCOlympics_2020/comp_test/images/'

    df = pd.DataFrame(columns=['pt_j1', 'm_j1', 'eta_j1', 'phi_j1', 'E_j1', 
                               'pt_j2', 'm_j2', 'eta_j2', 'phi_j2', 'E_j2',
                               'deltaeta', 'deltaphi',
                               'mEratio1', 'mEratio2',
                               'm_jj', 'pt_asym',
                               'deltaR1_sj12', 'deltaR1_sj13',
                               'deltaR1_sj14', 'deltaR1_sj23', 
                               'deltaR1_sj24', 'deltaR1_sj34',
                               'deltaR2_sj12', 'deltaR2_sj13',
                               'deltaR2_sj14', 'deltaR2_sj23', 
                               'deltaR2_sj24', 'deltaR2_sj34',
                               'n_subjets1','n_subjets2', 
                               'event_idx','img_name',
                               'P_BG','P_SIG'])

    for i in tqdm(range(events_combined.keys()[0],events_combined.keys()[-1]+1), 
                  total=len(batch)):

        pseudojets_input = np.zeros(len([x for x in events_combined[i][::3] if x > 0]), dtype=DTYPE_PTEPM)
        for j in range(700): #700 due to the way the jets are padded
            if (events_combined[i][j*3]>0):
                pseudojets_input[j]['pT'] = events_combined[i][j*3]
                pseudojets_input[j]['eta'] = events_combined[i][j*3+1]
                pseudojets_input[j]['phi'] = events_combined[i][j*3+2]
                pass
        weight_pt = pseudojets_input[:]['pT']/np.max(pseudojets_input[:]['pT'])

        DetectorImage(path=img_base_path, 
                  image_size=224,pixel_size=224).pseudojets_images(pseudojets_input, 
                                                                   weight=weight_pt, idx=i)

        img_name = DetectorImage.image_name(i)

        img_pth = os.path.join(img_base_path, img_name)

        img_test = open_image(img_pth)

        _, _, outputs = learn.predict(img_test)

        outputs = outputs.numpy()

        P_BG = outputs[0]
        P_SIG = outputs[1]

        os.remove(img_pth) #remove images after get the predictions to save space on disk :)

        sequence = cluster(pseudojets_input, R=1.0, algo='antikt')
        jets = sequence.inclusive_jets(500)

        pt_j1 = jets[0].pt
        m_j1 = np.abs(jets[0].mass)
        eta_j1 = jets[0].eta
        phi_j1 = jets[0].phi
        E_j1 = jets[0].e
        px_j1 = jets[0].px
        py_j1 = jets[0].py
        pz_j1 = jets[0].pz

        try:
            pt_j2 = jets[1].pt
            m_j2 = np.abs(jets[1].mass)
            eta_j2 = jets[1].eta
            phi_j2 = jets[1].phi
            E_j2 = jets[1].e
            px_j2 = jets[1].px
            py_j2 = jets[1].py
            pz_j2 = jets[1].pz

        except IndexError:
            pt_j2 = 0.
            m_j2 = 0.
            eta_j2 = 0.
            phi_j2 = 0.
            E_j2 = 0.
            px_j2 = 0.
            py_j2 = 0.
            pz_j2 = 0.

        deltaeta = np.abs(eta_j1 - eta_j2)
        deltaphi = comp_delta_phi(phi_j1,phi_j2)

        mEratio1 = m_j1/E_j1
        try:
            mEratio2 = m_j2/E_j2
        except ZeroDivisionError:
            mEratio2 = 0.

        E = E_j1+E_j2
        px = px_j1+px_j2
        py = py_j1+py_j2
        pz = pz_j1+pz_j2
        m_jj = (E**2-px**2-py**2-pz**2)**0.5

        pt_asym = (pt_j1 - pt_j2)/(pt_j1 + pt_j2)

        cluster_sj1 = cluster(jets[0].constituents_array(), R=0.4,p=1)
        sj1 = cluster_sj1.inclusive_jets(20)
        n_subjets1 = len(sj1)


        deltaR_list1 = delta_R_sub(sj1,4)
        deltaR1_sj12 = deltaR_list1[0]
        deltaR1_sj13 = deltaR_list1[1]
        deltaR1_sj14 = deltaR_list1[2]
        deltaR1_sj23 = deltaR_list1[3]
        deltaR1_sj24 = deltaR_list1[4]
        deltaR1_sj34 = deltaR_list1[5]

        try:
            cluster_sj2 = cluster(jets[1].constituents_array(), R=0.4,p=1)
            sj2 = cluster_sj2.inclusive_jets(20)
            n_subjets2 = len(sj2)

            deltaR_list2 = delta_R_sub(sj2,4)
            deltaR2_sj12 = deltaR_list2[0]
            deltaR2_sj13 = deltaR_list2[1]
            deltaR2_sj14 = deltaR_list2[2]
            deltaR2_sj23 = deltaR_list2[3]
            deltaR2_sj24 = deltaR_list2[4]
            deltaR2_sj34 = deltaR_list2[5]
        except IndexError:
            n_subjets2 = int(0)
            deltaR2_sj12 = 0.
            deltaR2_sj13 = 0.
            deltaR2_sj14 = 0.
            deltaR2_sj23 = 0.
            deltaR2_sj24 = 0.
            deltaR2_sj34 = 0.

        #make the pandas entry
        entry = pd.DataFrame([[pt_j1, m_j1, eta_j1, phi_j1, E_j1, 
                               pt_j2, m_j2, eta_j2, phi_j2, E_j2,
                               deltaeta, deltaphi,
                               mEratio1, mEratio2,
                               m_jj, pt_asym,
                               deltaR1_sj12, deltaR1_sj13,
                               deltaR1_sj14, deltaR1_sj23, 
                               deltaR1_sj24, deltaR1_sj34,
                               deltaR2_sj12, deltaR2_sj13,
                               deltaR2_sj14, deltaR2_sj23, 
                               deltaR2_sj24, deltaR2_sj34,
                               n_subjets1,n_subjets2, 
                               i, img_name, 
                               P_BG, P_SIG]],
                             columns=['pt_j1', 'm_j1', 'eta_j1', 'phi_j1', 'E_j1', 
                                      'pt_j2', 'm_j2', 'eta_j2', 'phi_j2', 'E_j2',
                                      'deltaeta', 'deltaphi',
                                      'mEratio1', 'mEratio2',
                                      'm_jj', 'pt_asym',
                                      'deltaR1_sj12', 'deltaR1_sj13',
                                      'deltaR1_sj14', 'deltaR1_sj23', 
                                      'deltaR1_sj24', 'deltaR1_sj34',
                                      'deltaR2_sj12', 'deltaR2_sj13',
                                      'deltaR2_sj14', 'deltaR2_sj23', 
                                      'deltaR2_sj24', 'deltaR2_sj34',
                                      'n_subjets1','n_subjets2', 
                                      'event_idx', 'img_name',
                                      'P_BG','P_SIG'])

        #appending the entry (row) into the main dataframe
        df = df.append(entry)        

    df.to_csv('/home/felipe/LHCOlympics_2020/comp_test/blackbox_batch_{}.csv'.format(batch_idx), sep=',', index=False)
    batch_idx += 1
    gc.collect()

In [ ]:
df[['P_SIG','P_BG']].describe()